In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from pathlib import Path
import random
import re

import matplotlib.pyplot as plt
from sklearn.cluster import k_means

random.seed(0)

In [2]:
import sys

sys.path.append("../src")

In [3]:
from core import SITE_NAMES
from calls import plot_call_features, compute_features, call_extraction

['/Users/adityakrishna/duty-cycle-investigation/daily_notebook', '/Users/adityakrishna/miniconda3/envs/dc-study/lib/python311.zip', '/Users/adityakrishna/miniconda3/envs/dc-study/lib/python3.11', '/Users/adityakrishna/miniconda3/envs/dc-study/lib/python3.11/lib-dynload', '', '/Users/adityakrishna/miniconda3/envs/dc-study/lib/python3.11/site-packages', '../src', '/Users/adityakrishna/duty-cycle-investigation/daily_notebook/../src/calls', '/Users/adityakrishna/duty-cycle-investigation/daily_notebook/../src', '/Users/adityakrishna/duty-cycle-investigation/daily_notebook/../src/bout', '/Users/adityakrishna/duty-cycle-investigation/daily_notebook/../src']


In [4]:
from core import SITE_NAMES, EXAMPLE_FILES_from_LOCATIONS, EXAMPLE_FILES_to_FILEPATHS, EXAMPLE_FILES_to_DETECTIONS, FREQ_GROUPS
import bout
import activity.activity_assembly as actvt

from cli import get_file_paths

In [5]:
cfg= dict()
cfg['site'] = 'Telephone'
cfg['recording_start'] = '01:00'
cfg['recording_end'] = '15:00'
cfg['percent_threshold_for_snr'] = 0.01
cfg['use_bouts'] = False
cfg['use_file'] = True

In [6]:
bucket_for_location = []
calls_sampled_from_location = pd.DataFrame()
location_sum_df, data_params = call_extraction.get_params_relevant_to_data_at_location(cfg)
location_sum_df

Searching for files from Telephone Field
Will be looking at 1075 files from Telephone Field


,index_in_file,freq_group,ref_time,call_start_time,call_end_time,start_time,end_time,low_freq,high_freq,event,class,class_prob,det_prob,individual,input_file,Site name,Recover Folder,SD Card,File Duration
0,0,HF2,2022-07-23 04:28:54.890500000,2022-07-23 04:28:54.890500000,2022-07-23 04:28:54.897500000,1734.8905,1734.8975,45234.0,55042.0,Echolocation,Pipistrellus pipistrellus,0.514,0.542,-1,/mnt/ubna_data_01_mir/recover-20220725/UBNA_01...,Telephone Field,recover-20220725,010,1795.0
1,1,HF2,2022-07-23 04:28:55.060500000,2022-07-23 04:28:55.060500000,2022-07-23 04:28:55.066600000,1735.0605,1735.0666,44375.0,57230.0,Echolocation,Pipistrellus pipistrellus,0.575,0.623,-1,/mnt/ubna_data_01_mir/recover-20220725/UBNA_01...,Telephone Field,recover-20220725,010,1795.0
2,2,HF2,2022-07-23 04:28:55.232500000,2022-07-23 04:28:55.232500000,2022-07-23 04:28:55.238400000,1735.2325,1735.2384,45234.0,54507.0,Echolocation,Pipistrellus pipistrellus,0.586,0.606,-1,/mnt/ubna_data_01_mir/recover-20220725/UBNA_01...,Telephone Field,recover-20220725,010,1795.0
3,3,HF2,2022-07-23 04:28:55.317500000,2022-07-23 04:28:55.317500000,2022-07-23 04:28:55.324800000,1735.3175,1735.3248,44375.0,50414.0,Echolocation,Pipistrellus pipistrellus,0.486,0.539,-1,/mnt/ubna_data_01_mir/recover-20220725/UBNA_01...,Telephone Field,recover-20220725,010,1795.0
4,4,HF2,2022-07-23 04:28:55.401500000,2022-07-23 04:28:55.401500000,2022-07-23 04:28:55.406800000,1735.4015,1735.4068,44375.0,54473.0,Echolocation,Pipistrellus pipistrellus,0.570,0.598,-1,/mnt/ubna_data_01_mir/recover-20220725/UBNA_01...,Telephone Field,recover-20220725,010,1795.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990978,633,HF2,2022-10-17 13:46:18.718500000,2022-10-17 13:46:18.718500000,2022-10-17 13:46:18.723900000,978.7185,978.7239,46093.0,55478.0,Echolocation,Pipistrellus pipistrellus,0.535,0.572,-1,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,007,NaN
990979,634,LF1,2022-10-17 13:49:58.244500000,2022-10-17 13:49:58.244500000,2022-10-17 13:49:58.254500000,1198.2445,1198.2545,24609.0,30880.0,Echolocation,Eptesicus serotinus,0.253,0.503,-1,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,007,NaN
990980,635,LF1,2022-10-17 13:49:59.893500000,2022-10-17 13:49:59.893500000,2022-10-17 13:49:59.909600000,1199.8935,1199.9096,22890.0,27197.0,Echolocation,Nyctalus leisleri,0.450,0.542,-1,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,007,NaN
990981,636,LF1,2022-10-17 13:50:00.414500000,2022-10-17 13:50:00.414500000,2022-10-17 13:50:00.428500000,1200.4145,1200.4285,22031.0,27346.0,Echolocation,Nyctalus leisleri,0.437,0.624,-1,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,007,NaN


In [7]:
csv_files_for_location = sorted(list(Path(f'../data/raw/{data_params["site_tag"]}').glob(pattern='*.csv')))
csv_files_for_location

[PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_010000.csv'),
 PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_013000.csv'),
 PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_020000.csv'),
 PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_023000.csv'),
 PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_030000.csv'),
 PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_033000.csv'),
 PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_040000.csv'),
 PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_043000.csv'),
 PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_050000.csv'),
 PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_053000.csv'),
 PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_060000.csv'),
 PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_063000.csv'),
 PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_070000.csv'),
 PosixPath('../data/raw/Telephone/bd2__Telephone_20220723_073000

In [8]:
data_params['bout_params']

{'site_key': 'Telephone',
 'HF2_bci': 1532.5273574275554,
 'HF1_bci': 1643.554173322591,
 'LF1_bci': 1204.7982742687193}

In [9]:
data_params['good_audio_files']

array(['/mnt/ubna_data_01_mir/recover-20220725/UBNA_010/20220723_040000.WAV',
       '/mnt/ubna_data_01_mir/recover-20220725/UBNA_010/20220723_043000.WAV',
       '/mnt/ubna_data_01_mir/recover-20220725/UBNA_010/20220723_050000.WAV',
       ...,
       '/mnt/ubna_data_02_mir/recover-20221017/UBNA_007/20221017_123000.WAV',
       '/mnt/ubna_data_02_mir/recover-20221017/UBNA_007/20221017_130000.WAV',
       '/mnt/ubna_data_02_mir/recover-20221017/UBNA_007/20221017_133000.WAV'],
      dtype=object)

In [10]:
filepath = data_params['good_audio_files'][-1]
data_params['audio_file'] = Path(filepath)
filename = data_params['audio_file'].name.split('.')[0]
csv_path = Path(f'../data/raw/{data_params["site_tag"]}/bd2__{data_params["site_tag"]}_{filename}.csv')
print(f'Looking at {filepath} with detection file: {csv_path}')
data_params['csv_file'] = csv_path
data_params['percent_threshold_for_snr'] = cfg['percent_threshold_for_snr']
data_params['use_bouts'] = cfg['use_bouts']
data_params['use_file'] = cfg['use_file']
data_params

Looking at /mnt/ubna_data_02_mir/recover-20221017/UBNA_007/20221017_133000.WAV with detection file: ../data/raw/Telephone/bd2__Telephone_20221017_133000.csv


{'type_tag': '',
 'cur_dc_tag': '1800of1800',
 'site_tag': 'Telephone',
 'site_name': 'Telephone Field',
 'good_audio_files': array(['/mnt/ubna_data_01_mir/recover-20220725/UBNA_010/20220723_040000.WAV',
        '/mnt/ubna_data_01_mir/recover-20220725/UBNA_010/20220723_043000.WAV',
        '/mnt/ubna_data_01_mir/recover-20220725/UBNA_010/20220723_050000.WAV',
        ...,
        '/mnt/ubna_data_02_mir/recover-20221017/UBNA_007/20221017_123000.WAV',
        '/mnt/ubna_data_02_mir/recover-20221017/UBNA_007/20221017_130000.WAV',
        '/mnt/ubna_data_02_mir/recover-20221017/UBNA_007/20221017_133000.WAV'],
       dtype=object),
 'bout_params': {'site_key': 'Telephone',
  'HF2_bci': 1532.5273574275554,
  'HF1_bci': 1643.554173322591,
  'LF1_bci': 1204.7982742687193},
 'audio_file': PosixPath('/mnt/ubna_data_02_mir/recover-20221017/UBNA_007/20221017_133000.WAV'),
 'csv_file': PosixPath('../data/raw/Telephone/bd2__Telephone_20221017_133000.csv'),
 'percent_threshold_for_snr': 0.01,
 'use_b

In [11]:
csv_path = Path(data_params['csv_file'])

bd2_predictions = actvt.assemble_single_bd2_output(csv_path, data_params)
groups_in_preds = bd2_predictions['freq_group'].unique()
valid_group_in_preds = np.logical_or(np.logical_or('LF1' in groups_in_preds, 'HF1' in groups_in_preds), 'HF2' in groups_in_preds)
print(f"Groups found in this file: {bd2_predictions['freq_group'].unique()}, valid? {valid_group_in_preds}")

Groups found in this file: ['HF2' '' 'HF1' 'LF1'], valid? True


In [12]:
bd2_predictions

,freq_group,ref_time,call_start_time,call_end_time,start_time,end_time,low_freq,high_freq,class,class_prob,det_prob,individual,event,input_file,Site name,Recover Folder,SD Card
0,HF2,2022-10-17 13:30:15.817500000,2022-10-17 13:30:15.817500000,2022-10-17 13:30:15.823600,15.8175,15.8236,46093.0,53917.0,Pipistrellus pipistrellus,0.463,0.518,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
1,HF2,2022-10-17 13:30:16.245500000,2022-10-17 13:30:16.245500000,2022-10-17 13:30:16.251600,16.2455,16.2516,46093.0,52657.0,Pipistrellus pipistrellus,0.541,0.579,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
2,HF2,2022-10-17 13:30:16.436500000,2022-10-17 13:30:16.436500000,2022-10-17 13:30:16.442300,16.4365,16.4423,46953.0,56186.0,Pipistrellus pipistrellus,0.584,0.614,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
3,HF2,2022-10-17 13:30:16.527499999,2022-10-17 13:30:16.527499999,2022-10-17 13:30:16.533100,16.5275,16.5331,45234.0,54887.0,Pipistrellus pipistrellus,0.525,0.551,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
4,HF2,2022-10-17 13:30:17.031500000,2022-10-17 13:30:17.031500000,2022-10-17 13:30:17.036300,17.0315,17.0363,45234.0,60188.0,Pipistrellus pipistrellus,0.371,0.537,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,HF2,2022-10-17 13:46:18.718500000,2022-10-17 13:46:18.718500000,2022-10-17 13:46:18.723900,978.7185,978.7239,46093.0,55478.0,Pipistrellus pipistrellus,0.535,0.572,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
634,LF1,2022-10-17 13:49:58.244500000,2022-10-17 13:49:58.244500000,2022-10-17 13:49:58.254500,1198.2445,1198.2545,24609.0,30880.0,Eptesicus serotinus,0.253,0.503,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
635,LF1,2022-10-17 13:49:59.893500000,2022-10-17 13:49:59.893500000,2022-10-17 13:49:59.909600,1199.8935,1199.9096,22890.0,27197.0,Nyctalus leisleri,0.450,0.542,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
636,LF1,2022-10-17 13:50:00.414500000,2022-10-17 13:50:00.414500000,2022-10-17 13:50:00.428500,1200.4145,1200.4285,22031.0,27346.0,Nyctalus leisleri,0.437,0.624,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7


In [14]:
location_sum_df['input_file'] = call_extraction.relabel_drivenames_to_mirrors(location_sum_df['input_file'].copy())

In [17]:
data_params['audio_file']

PosixPath('/mnt/ubna_data_02_mir/recover-20221017/UBNA_007/20221017_133000.WAV')

In [21]:
bd2_preds = location_sum_df.loc[location_sum_df['input_file']==str(data_params['audio_file'])].copy()

In [22]:
bout_metrics = call_extraction.get_bout_metrics_from_single_bd2_output(bd2_preds, data_params)
bout_metrics.reset_index(inplace=True)
if 'index' in bout_metrics.columns:
    bout_metrics.drop(columns='index', inplace=True)
    
file_path = Path(data_params['audio_file'])
print(f'{len(bd2_preds)} calls in this file: {file_path.name}')

638 calls in this file: 20221017_133000.WAV


In [23]:
bout_metrics

,freq_group,start_time_of_bout,end_time_of_bout,start_time_wrt_ref,end_time_wrt_ref,start_time,end_time,low_freq,high_freq,number_of_dets,bout_duration,bout_duration_in_secs
0,HF2,2022-10-17 13:30:15.817500000,2022-10-17 13:30:20.741200,15.8175,20.7412,15.8175,20.7412,42656.0,76520.0,29,0 days 00:00:04.923700,4.9237
1,HF2,2022-10-17 13:30:28.038500000,2022-10-17 13:30:31.213800,28.0385,31.2138,28.0385,31.2138,41796.0,79567.0,10,0 days 00:00:03.175300,3.1753
2,HF2,2022-10-17 13:32:19.534500000,2022-10-17 13:32:19.799200,139.5345,139.7992,139.5345,139.7992,45234.0,60255.0,3,0 days 00:00:00.264700,0.2647
3,HF2,2022-10-17 13:32:24.659500000,2022-10-17 13:32:24.746100,144.6595,144.7461,144.6595,144.7461,45234.0,56409.0,2,0 days 00:00:00.086600,0.0866
4,HF2,2022-10-17 13:32:29.675500000,2022-10-17 13:32:30.243000,149.6755,150.2430,149.6755,150.2430,44375.0,59348.0,3,0 days 00:00:00.567500,0.5675
5,HF2,2022-10-17 13:32:41.687500000,2022-10-17 13:32:42.324400,161.6875,162.3244,161.6875,162.3244,44375.0,56344.0,8,0 days 00:00:00.636900,0.6369
6,HF2,2022-10-17 13:32:44.977500000,2022-10-17 13:32:45.482100,164.9775,165.4821,164.9775,165.4821,44375.0,60082.0,4,0 days 00:00:00.504600,0.5046
7,HF2,2022-10-17 13:33:23.528499999,2022-10-17 13:33:24.074400,203.5285,204.0744,203.5285,204.0744,44375.0,58488.0,3,0 days 00:00:00.545900001,0.5459
8,HF2,2022-10-17 13:33:33.076500000,2022-10-17 13:33:33.756300,213.0765,213.7563,213.0765,213.7563,45234.0,59156.0,7,0 days 00:00:00.679800,0.6798
9,HF2,2022-10-17 13:34:10.881500000,2022-10-17 13:34:11.607100,250.8815,251.6071,250.8815,251.6071,42656.0,67381.0,5,0 days 00:00:00.725600,0.7256


In [29]:
row = bout_metrics.iloc[1]
group = row['freq_group']
freq_group = bd2_predictions.loc[bd2_predictions['freq_group']==group]
bat_bout = freq_group.loc[(freq_group['start_time']>=row['start_time'])&(freq_group['end_time']<=row['end_time'])].copy()

In [30]:
bat_bout

,freq_group,ref_time,call_start_time,call_end_time,start_time,end_time,low_freq,high_freq,class,class_prob,det_prob,individual,event,input_file,Site name,Recover Folder,SD Card
31,HF2,2022-10-17 13:30:28.038500,2022-10-17 13:30:28.038500,2022-10-17 13:30:28.042800,28.0385,28.0428,52109.0,62854.0,Pipistrellus pipistrellus,0.464,0.519,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
32,HF2,2022-10-17 13:30:29.260500,2022-10-17 13:30:29.260500,2022-10-17 13:30:29.265400,29.2605,29.2654,42656.0,66103.0,Pipistrellus pipistrellus,0.534,0.716,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
33,HF2,2022-10-17 13:30:29.617500,2022-10-17 13:30:29.617500,2022-10-17 13:30:29.622600,29.6175,29.6226,46953.0,59305.0,Pipistrellus pipistrellus,0.545,0.602,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
34,HF2,2022-10-17 13:30:29.960500,2022-10-17 13:30:29.960500,2022-10-17 13:30:29.964700,29.9605,29.9647,43515.0,73052.0,Myotis alcathoe,0.248,0.523,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
36,HF2,2022-10-17 13:30:30.371500,2022-10-17 13:30:30.371500,2022-10-17 13:30:30.376500,30.3715,30.3765,45234.0,58338.0,Pipistrellus pipistrellus,0.575,0.652,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
37,HF2,2022-10-17 13:30:30.537500,2022-10-17 13:30:30.537500,2022-10-17 13:30:30.541500,30.5375,30.5415,42656.0,58044.0,Myotis alcathoe,0.116,0.509,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
38,HF2,2022-10-17 13:30:30.619500,2022-10-17 13:30:30.619500,2022-10-17 13:30:30.623500,30.6195,30.6235,42656.0,66830.0,Myotis alcathoe,0.294,0.507,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
39,HF2,2022-10-17 13:30:30.702500,2022-10-17 13:30:30.702500,2022-10-17 13:30:30.706300,30.7025,30.7063,42656.0,77810.0,Myotis alcathoe,0.418,0.576,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
40,HF2,2022-10-17 13:30:31.063500,2022-10-17 13:30:31.063500,2022-10-17 13:30:31.066900,31.0635,31.0669,41796.0,79567.0,Myotis alcathoe,0.339,0.589,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
41,HF2,2022-10-17 13:30:31.210500,2022-10-17 13:30:31.210500,2022-10-17 13:30:31.213800,31.2105,31.2138,46093.0,72679.0,Myotis alcathoe,0.323,0.501,-1,Echolocation,/mnt/ubna_data_02/recover-20221017/UBNA_007/20...,Telephone Field,recover-20221017,7
